In [1]:
import pandas as pd
import numpy as np
import folium

from sklearn.preprocessing import normalize
from sklearn.base import ClusterMixin
from sklearn.cluster import KMeans, SpectralClustering


np.random.seed(777)

In [2]:
class GraphClustering(ClusterMixin):
    def __init__(self, n_clusters=8, n_components=None, **kwargs):
        '''
        Spectral clustering algorithm
        param n_clusters: number of clusters to form
        param n_components: number of eigenvectors to use
        '''

        if n_components is None:
            n_components = n_clusters

        self.n_components = n_components
        self.kmeans = KMeans(n_clusters=n_clusters, **kwargs)

    def fit_predict(self, X, y=None):
        '''
        Perform spectral clustering from graph adjacency matrix
        and return vertex labels.
        param X: (n_samples, n_samples) - graph adjacency matrix
        return: (n_samples, ) - vertex labels
        '''

        eigenvectors = self._generate_eigenvectors(X)
        labels = self.kmeans.fit_predict(eigenvectors[:, 1:])
        return labels

    def _generate_eigenvectors(self, X):
        '''
        Compute eigenvectors for spectral clustering
        param X: (n_samples, n_samples) - graph adjacency matrix
        return: (n_samples, n_components) - eigenvectors
        '''
        diagonal_matrix = np.diag(np.sum(X, axis=1))
        laplace_matrix = diagonal_matrix - X
        eig_val, eig_vec = np.linalg.eig(laplace_matrix)
        eig_val = np.real(eig_val)
        eig_vec = np.real(eig_vec) 
        eig_vec = eig_vec[:, np.argsort(eig_val)]
        eig_val = eig_val[np.argsort(eig_val)]
        zero_eig_val_num = eig_val[np.abs(eig_val)<1e-10].shape[0]
        
        return eig_vec[:, :self.n_components]

In [3]:
# a = np.array([[1,2,3], [6,7,93], [4,7,6]])
# graph_clustering_test = GraphClustering(n_clusters=10)
# labels_test = graph_clustering_test.fit_predict(a)
# labels_test


In [4]:
n_blocks, n_vertices = 10, 1000
block_vertices = n_vertices // n_blocks

X = np.zeros((n_vertices, n_vertices))
for i in range(0, n_vertices, block_vertices):
    X[i:i + block_vertices, i:i + block_vertices] = np.sqrt(i + 1)

# X = np.array(
# [[0., 1., 1., 0., 0., 1., 0., 0., 1., 1.],
#  [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
#  [1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
#  [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
#  [0., 0., 0., 1., 0., 1., 0., 0., 0., 0.],
#  [1., 0., 0., 1., 1., 0., 1., 1., 0., 0.],
#  [0., 0., 0., 0., 0., 1., 0., 1., 0., 0.],
#  [0., 0., 0., 0., 0., 1., 1., 0., 0., 0.],
#  [1., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
#  [1., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])
    
graph_clustering = GraphClustering(n_clusters=n_blocks)
labels_graph = graph_clustering.fit_predict(X)
print(np.bincount(labels_graph))

# spectral_clustering = SpectralClustering(n_clusters=n_blocks, affinity='precomputed')
# labels_spectral = spectral_clustering.fit_predict(X)
# print(spectral_clustering.affinity_matrix_)
# print(np.bincount(labels_spectral))

true_labels = np.zeros(n_vertices, dtype=np.int32)
for i in range(0, n_vertices, block_vertices):
    true_labels[i:i + block_vertices] = labels_graph[i]

assert labels_graph.shape == (n_vertices, )
assert np.all(np.bincount(labels_graph) == np.full(n_blocks, block_vertices))
assert np.all(labels_graph == true_labels)

[100 100 100 100 100 100 100 100 100 100]


data = pd.read_excel('data/City surface public transport stops.xlsx')
data = data[data.AdmArea_en == "Czentral`ny'j administrativny'j okrug"]
data = data.reset_index()
data.head()

map = folium.Map([55.75215, 37.61819], zoom_start=12)
for id, row in data.iterrows():
    folium.Circle([row.Latitude_WGS84_en, row.Longitude_WGS84_en],
                  radius=10).add_to(map)
map

def get_routes(data):
    '''
    Accumulate routes from raw data
    param data: pd.DataFrame - public transport stops data
    return: dict - unsorted stops ids for each route,
                   e.g. routes['A1'] = [356, 641, 190]
    '''

    # YOUR CODE HERE ‿︵‿︵ヽ(°□° )ノ︵‿︵‿
    raise NotImplementedError


def sort_routes(data, routes):
    '''
    Sort routes according to the proposed algorithm
    param data: pd.DataFrame - public transport stops data
    param routes: dict - unsorted stops ids for each route
    return: dict - sorted stops ids for each route
    '''

    # YOUR CODE HERE ‿︵‿︵ヽ(°□° )ノ︵‿︵‿
    raise NotImplementedError


def get_adjacency_matrix(data, sorted_routes):
    '''
    Compute adjacency matrix for sorted routes
    param data: pd.DataFrame - public transport stops data
    param sorted_routes: dict - sorted stops ids for each route
    return: (n_samples, n_samples) - graph adjacency matrix
    '''

    # YOUR CODE HERE ‿︵‿︵ヽ(°□° )ノ︵‿︵‿
    raise NotImplementedError

routes = get_routes(data)
sorted_routes = sort_routes(data, routes)
adjacency_matrix = get_adjacency_matrix(data, sorted_routes)

map = folium.Map([55.75215, 37.61819], zoom_start=12)
for route_id in np.random.choice(list(sorted_routes.keys()), size=5):
    coords = data.loc[
        sorted_routes[route_id],
        ['Latitude_WGS84_en', 'Longitude_WGS84_en']
    ].values.tolist()
    folium.vector_layers.PolyLine(coords).add_to(map)

map

def draw_clustered_map(data, labels):
    '''
    Create map with coloured clusters
    param data: pd.DataFrame - public transport stops data
    param labels: (n_samples, ) - cluster labels for each stop
    return: folium.Map - map with coloured clusters
    '''

    # YOUR CODE HERE ‿︵‿︵ヽ(°□° )ノ︵‿︵‿
    raise NotImplementedError